In [4]:
# Import all necessary libraries.
import os
import cv2
import sys
import json
import matplotlib.image as mpimg
import warnings
from collections import Counter

from _paths import nomeroff_net_dir
from nomeroff_net.pipes.number_plate_classificators.options_detector import OptionsDetector

In [2]:
warnings.filterwarnings('ignore')

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

NPOptionsNet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [22]:
def optionChecker(datasetDir, imgFormat="png", partSize=1):
    ann = "ann"
    img = "img"
    annDir = os.path.join(datasetDir, ann)
    imgDir = os.path.join(datasetDir, img)
    
    imgFnames         = []
    nottedRegions     = []
    nottedcount_lines = []
    nottedStates      = []
    zones             = []
    annFnames         = []
    annData           = []
    i = 0
    counter = Counter()
    for dir_name, subdir_list, file_list in os.walk(annDir):
        for fname in file_list:
            ann_path = os.path.join(annDir, fname)
            
            i += 1
            annFnames.append(ann_path)
            with open(ann_path) as jsonR:
                data = json.load(jsonR)
            img_name = data["name"]
            annData.append(data)
            
            img_path = os.path.join(imgDir, "{}.{}".format(img_name, imgFormat))
            zones.append(cv2.cvtColor(mpimg.imread(img_path), cv2.COLOR_RGB2BGR))
            
            imgFnames.append(img_path)
            nottedRegions.append(data["region_id"])
            nottedStates.append(data["state_id"])
            nottedcount_lines.append(data["count_lines"])
            if i >= partSize:
                # find standart
                region_ids, count_lines = optionsDetector.predict(zones)
                for regionId, zone, nottedRegion, nottedState, imgFname, annFname, annItem, nottedCountL, countL in zip(region_ids, 
                                                                                                                        zones, 
                                                                                                                        nottedRegions, 
                                                                                                                        nottedStates, 
                                                                                                                        imgFnames, 
                                                                                                                        annFnames, 
                                                                                                                        annData, 
                                                                                                                        count_lines, 
                                                                                                                        nottedcount_lines):
                    
                    # region
                    bad_region=False
                    if int(regionId) != int(nottedRegion):
                        bad_region=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['regionPredicted'] = int(regionId)
                        print("REGION NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(regionId))
                        print("ANNOTATED: {}".format(nottedRegion))
                        counter["BAD_REGION"] += 1
                    else:
                        counter["GOOD_REGION"] += 1
                    
                    # count
                    bad_count=False
                    if int(countL) != int(nottedCountL):
                        bad_count=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['countPredicted'] = int(countL)
                        print("COUNT LINES NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(countL))
                        print("ANNOTATED: {}".format(nottedCountL))
                        counter["BAD_COUNT"] += 1
                    else:
                        counter["GOOD_COUNT"] += 1

                    if 'moderation' in annItem and (bad_region or bad_count):
                        with open(annFname, "w") as jsonW:
                            annItem['moderation']['isModerated'] = 0
                            json.dump(annItem, jsonW)                    
                    
                imgFnames        = []
                nottedRegions    = []
                nottedcount_lines = []
                nottedStates     = []
                zones            = []
                annFnames        = []
                annData          = []
    return counter

In [23]:
valDir = os.path.join(nomeroff_net_dir, "./data/dataset/OptionsDetector/numberplate_options_example/train")
optionChecker(valDir)

Counter({'GOOD_REGION': 1, 'GOOD_COUNT': 1})